This kernel not including EDA because this kernel is too long and our main goal is building prediction model.
For features,We just collected required features.

# 1-Preprocessing
    We do this job by seperating classes or files all code.
    Then,We use all these methods for build our model.

In [ ]:
!mkdir labelencoder
!mkdir onehotencoder
!mkdir inputs
!mkdir standartScaler
!mkdir test_data
!mkdir specific
!mkdir model
!mkdir model/teams

mkdir: cannot create directory ‘labelencoder’: File exists
mkdir: cannot create directory ‘onehotencoder’: File exists
mkdir: cannot create directory ‘inputs’: File exists
mkdir: cannot create directory ‘standartScaler’: File exists
mkdir: cannot create directory ‘test_data’: File exists
mkdir: cannot create directory ‘specific’: File exists
mkdir: cannot create directory ‘model’: File exists
mkdir: cannot create directory ‘model/teams’: File exists


In [ ]:
#Importing Libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def get_golden_feature_list(df,threshold=.5):
    df_numeric = df.select_dtypes(include=["float64","int64"])
    df_corr = df_numeric.corr()
    df_corr = df_corr.iloc[2,:-1]
    golden_features_list = df_corr[abs(df_corr) >= threshold].sort_values(ascending=False)
    return golden_features_list


##### *get_golden_feature_list() : getting muchcorrelated features with total points which greater equal than threshold*

In [ ]:
def get_worst_feature_list(df,threshold=.5):
    df_numeric = df.select_dtypes(include=["float64","int64"])
    df_corr = df_numeric.corr()
    df_corr = df_corr.iloc[2,:-1]
    golden_features_list = df_corr[abs(df_corr) <= threshold].sort_values(ascending=False)
    return golden_features_list


##### *get_worst_feature_list() : getting much uncorrelated features with total points which lower equal than threshold*

In [ ]:
# Clean Up Columns Names
def remove_whitespaces_in_df_columns(df):
    df.columns = df.columns.str.replace(' ', '')
    return df.columns

In [ ]:
# Get Columns Names For All Data
def get_column_names():
    """
    :return: Column names of Data.csv
    """
    feature_names_of_players = ["PLAYER_NAME", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA",
                                "FT_PCT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PTS", "PLUS_MINUS"]

    feature_names_of_matchups = ["gameId", "teamAbbr", "opptAbbr", "rslt", "teamMin", "teamPTS", "teamPTS1", "teamPTS2",
                                 "teamPTS3", "teamPTS4", "opptPTS",
                                 "opptPTS1", "opptPTS2", "opptPTS3", "opptPTS4",
                                 "teamFGM", "teamFGA", "teamFG", "team3PM", "team3PA", "team3PCT", "teamFTM", "teamFTA",
                                 "teamFTC", "teamORB", "teamDRB", "teamREB", "teamAST", "teamSTL"
        , "teamBLK", "teamTO", "teamPF", "team2P", "teamTS", "teamEFG", "teamPPS", "teamFIC", "teamFIC40", "teamOrtg",
                                 "teamDrtg", "teamPlay",
                                 "opptMin", "opptFGM", "opptFGA", "opptFG", "oppt3PM", "oppt3PA", "oppt3PCT", "opptFTM",
                                 "opptFTA", "opptFTC", "opptORB", "opptDRB", "opptREB", "opptAST", "opptSTL"
        , "opptBLK", "opptTO", "opptPF", "oppt2P", "opptTS", "opptEFG", "opptPPS", "opptFIC", "opptFIC40", "opptOrtg",
                                 "opptDrtg", "opptPlay", "poss", "pace"]

    team_features = ["gameId", "teamAbbr", "opptAbbr", "rslt", "teamMin", "teamPTS", "teamPTS1", "teamPTS2", "teamPTS3",
                     "teamPTS4", "opptPTS",
                     "opptPTS1", "opptPTS2", "opptPTS3", "opptPTS4",
                     "teamFGM", "teamFGA", "teamFG", "team3PM", "team3PA", "team3PCT", "teamFTM", "teamFTA", "teamFTC",
                     "teamORB", "teamDRB", "teamREB", "teamAST", "teamSTL"
        , "teamBLK", "teamTO", "teamPF", "team2P", "teamTS", "teamEFG", "teamPPS", "teamFIC", "teamFIC40", "teamOrtg",
                     "teamDrtg", "teamPlay"]
    team_features = team_features + feature_names_of_players * 11

    oppt_features = ["opptMin", "opptFGM", "opptFGA", "opptFG", "oppt3PM", "oppt3PA", "oppt3PCT", "opptFTM", "opptFTA",
                     "opptFTC", "opptORB", "opptDRB", "opptREB", "opptAST", "opptSTL"
        , "opptBLK", "opptTO", "opptPF", "oppt2P", "opptTS", "opptEFG", "opptPPS", "opptFIC", "opptFIC40", "opptOrtg",
                     "opptDrtg", "opptPlay"]

    oppt_features = oppt_features + feature_names_of_players * 11

    last_features = ["poss", "LM_totalPoint","LM_dayOffset","pace"]

    feature_names_of_matchups = team_features + oppt_features + last_features
    return feature_names_of_matchups


In [ ]:
# Getting unique team names 
def get_unique_team_abbr(df):
    """

    :param df: Data.csv
    :return: Team Abbrs
    """
    return np.unique(df.teamAbbr.values)


In [ ]:
# Getting Unique Player List
def get_unique_player_list(df):
    """
    :param df: Data.csv
    :return: Player List
    """
    players = []
    for i in range(df.PLAYER_NAME.shape[0]):
        for j in range(df.PLAYER_NAME.shape[1]):
            players.append(df.PLAYER_NAME.iloc[i, j])

    unique_players = np.unique(players)
    return unique_players


In [ ]:
# Now,We write 3 methods for re-creating all player stats dataset.

# First one is creating empty dataset to fill with real stats.

def create_empty_player_stats(player_list):
    """

    :param player_list: Oyuncu Listesi
    :return: Oyuncu sayısı kadar row olan stats tablosu döndürür
    """
    col_names_for_player_stats = ["PLAYER_NAME", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA",
                                  "FT_PCT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PTS", "PLUS_MINUS"]
    index_for_player_stats = range(len(player_list))
    player_stats_df = pd.DataFrame(columns=col_names_for_player_stats, index=index_for_player_stats)
    return player_stats_df

# Second one is splitting main data and getting player stats

def get_all_players_stats_df(df):
    """

    :param df: Data.csv --> Dataframe
    :return:Bütün oyuncuların tüm maçlardaki verilerini döndürür
    """
    team_player_first_column_index = df.columns.get_loc("teamPlay") + 1
    team_player_last_column_index = df.columns.get_loc("opptMin")
    oppt_player_first_column_index = df.columns.get_loc("opptPlay") + 1
    oppt_player_last_column_index = df.columns.get_loc("poss")
    team_player_stats = df.iloc[:, team_player_first_column_index:team_player_last_column_index]
    oppt_player_stats = df.iloc[:, oppt_player_first_column_index:oppt_player_last_column_index]
    player_stats_conc = pd.concat([team_player_stats, oppt_player_stats], axis=0)
    player_stats_conc_splited = np.array_split(player_stats_conc, 11, axis=1)
    full_df = pd.concat([player_stats_conc_splited[0],
                         player_stats_conc_splited[1],
                         player_stats_conc_splited[2],
                         player_stats_conc_splited[3],
                         player_stats_conc_splited[4],
                         player_stats_conc_splited[5],
                         player_stats_conc_splited[6],
                         player_stats_conc_splited[7],
                         player_stats_conc_splited[8],
                         player_stats_conc_splited[9],
                         player_stats_conc_splited[10]], axis=0)
    return full_df

# Last one is filling empty dataframe with these datas
# Last x Match variable is how many match ago

def get_all_player_stats_last_x_match(all_player_stats_df,all_player_stats_empty_df,player_list,last_x_match=10):
    """

    :param all_player_stats_df: Maçlara göre olan bütün oyuncuların verileri
    :param all_player_stats_empty_df: Player sayısı kadar row olan boş veri tablosu
    :param player_list: Playerların listesi
    :param last_x_match: Son kaç maç olduğu
    :return: Bütün oyuncuların ortalama verileri
    """
    for i in range(len(player_list)):
        each = player_list[i]
        player_stats_each = all_player_stats_df[all_player_stats_df.PLAYER_NAME == each][-1:-last_x_match:-1]
        player_name = each
        mean_of_stats_this_player = player_stats_each.drop(columns=["PLAYER_NAME"], axis=1).mean()
        all_player_stats_empty_df.at[i, "PLAYER_NAME"] = player_name
        all_player_stats_empty_df.iloc[i, 1:] = mean_of_stats_this_player
    try :
        all_player_stats_empty_df.to_csv("inputs/all_player_stats.csv")
        print("CSV Saved To Input Folder name all_player_stats.csv")
    except Exception as e:
        print(e)
    return all_player_stats_empty_df


In [ ]:
# If you want to get just one player by name you can use this method
# All player stats df is dataframe which get_all_player_stats_last_x_match return
def get_player_stats_by_name(all_player_stats_df,player_name,last_x_match=10):
    """

    :param all_player_stats_df: Bütün playerların maçlara göre verileri
    :param player_name: Player ismi
    :param last_x_match: Son kaç maç olduğu
    :return: Playerın ortalama değerleri
    """

    player_stats_each = all_player_stats_df[all_player_stats_df.PLAYER_NAME == player_name][-1:-last_x_match:-1]
    mean_of_stats_this_player = player_stats_each.drop(columns=["PLAYER_NAME"], axis=1).mean()
    # all_player_stats_empty_df.at[i, "PLAYER_NAME"] = player_name
    # all_player_stats_empty_df.iloc[i, 1:] = mean_of_stats_this_player

    return player_name,mean_of_stats_this_player



In [ ]:
# Now we create dataframe which includes every teams matches with each others.
# df is main dataframe
def get_play_by_play_stats(df,last_x_match=10):
    """

    :param df: Data.csv
    :param last_x_match: Son kaç maç
    :return: Takımların diğer takımlarla olan maç istatistikleri
    """

    """preparing dropped columns"""

    teamAbbr_unique = get_unique_team_abbr(df)
    opptAbbr_unique = get_unique_team_abbr(df)
    coll = df.drop(df.columns[42:272],axis=1)
    coll = df.drop(df.columns[300:530],axis=1)
    coll = coll.drop("gameId",axis=1)
    df = coll

    play_by_play = pd.DataFrame(columns=df.columns, index=range(len(teamAbbr_unique) * len(teamAbbr_unique)))
    i = 0
    for each in teamAbbr_unique:
        for k in opptAbbr_unique:
            if each != k:
                if df[(df.teamAbbr == each) & (df.opptAbbr == k)].empty == False:
                    df_for_each = df[(df.teamAbbr == each) & (df.opptAbbr == k)][-1:-last_x_match:-1]

                    play_by_play_count = df_for_each["rslt"].value_counts()
                    # print(each,k)
                    # print(df[(df.teamAbbr == each) & (df.opptAbbr == k)])
                    if play_by_play_count.shape[0] == 1:
                        play_by_play_count = play_by_play_count[0] / (play_by_play_count[0]) * 100
                    else:
                        play_by_play_count = play_by_play_count[0] / (play_by_play_count[0] + play_by_play_count[1]) * 100
                    play_by_play.iloc[i, 3:] = df[(df.teamAbbr == each) & (df.opptAbbr == k)].iloc[:, 3:].mean()
                    play_by_play.loc[i, "teamAbbr"] = each
                    play_by_play.loc[i, "opptAbbr"] = k
                    play_by_play.at[i, "rslt"] = play_by_play_count

                    i += 1
                else:
                    pass
    try :
        play_by_play.to_csv("inputs/play_by_play.csv")
        print("CSV Saved To Input Folder name play_by_play.csv")
    except Exception as e:
        print(e)
    return play_by_play


In [ ]:
# If you want to use any match in your model you can create one row data with this method.

# teamAbbr = home Team
# opptAbbr = away Team
# data_csv_name = Main data path
# Player stats csv name = all_player_stats path
def get_test_data(teamAbbr,opptAbbr,homeplayers,awayplayers,data_csv_name = "/content/inputs/matches_w_player_stats.csv",player_stats_csv_name = "../input/all_player_stats.csv",last_x_match=10):
    df = pd.read_csv(data_csv_name)
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    all_players_stats_df = pd.read_csv(player_stats_csv_name,index_col=0)
    play_by_play = get_play_by_play_stats(df,last_x_match)
    test_data = play_by_play[(play_by_play["teamAbbr"] == teamAbbr) & (play_by_play["opptAbbr"] == opptAbbr)]
    home = []
    away = []
    feature_names_of_players = ["MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT", "FTM", "FTA",
                                "FT_PCT", "OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PTS", "PLUS_MINUS"]
    for each in homeplayers:
        player_name , mean_of_player = get_player_stats_by_name(all_player_stats_df=all_players_stats_df,player_name=each,last_x_match=last_x_match)
        mean_of_player = mean_of_player.tolist()
        home.append(mean_of_player)
    for each in awayplayers:
        player_name , mean_of_player = get_player_stats_by_name(all_player_stats_df=all_players_stats_df,player_name=each,last_x_match=last_x_match)
        mean_of_player = mean_of_player.tolist()
        away.append(mean_of_player)
    all_df = pd.DataFrame(columns=range(len(feature_names_of_players)*(len(homeplayers)+len(awayplayers))),index=range(1))
    start_offset = 0
    end_offset = len(feature_names_of_players)
    for i in range(len(homeplayers)):
        all_df.iloc[0,start_offset:end_offset] = home[i]
        start_offset = end_offset
        end_offset += 20
    start_offset = len(homeplayers) * len(feature_names_of_players)
    end_offset = start_offset + len(feature_names_of_players)
    for i in range(len(awayplayers)):
        all_df.iloc[0,start_offset:end_offset] = away[i]
        start_offset = end_offset
        end_offset += len(feature_names_of_players)
    all_df.columns = feature_names_of_players * (len(homeplayers) + len(awayplayers))
    test_data.reset_index(inplace=True)

    conc =  pd.concat([test_data,all_df],axis=1)
    conc.to_csv("test_data/"+teamAbbr+"vs"+opptAbbr+"_test_data.csv")
    conc.fillna(value=0,inplace=True)
    conc.drop("index",axis=1,inplace=True)
    return conc


### Now we preprocessed our data.Time to label encoding , one hot encoding , scaling.

In [ ]:
# Dropping un neccesary columns for training.
def drop_some_columns(df):
    df = df.drop(columns=["PLAYER_NAME"],axis=1)
    df = df.drop(columns=["gameId"],axis=1)
    return df


In [ ]:
# Label Encoding all data and saving label encoder.
def label_train_data(df):
    le = LabelEncoder()
    df["teamAbbr"] = le.fit_transform(df["teamAbbr"])
    keys = le.classes_
    values = le.transform(le.classes_)
    dictionary1 = dict(zip(keys, values))
    np.save('labelencoder/team_abbr.npy', le.classes_)
    df["opptAbbr"] = le.fit_transform(df["opptAbbr"])
    keys = le.classes_
    values = le.transform(le.classes_)
    np.save('labelencoder/oppt_abbr.npy', le.classes_)
    dictionary2 = dict(zip(keys, values))
    df["rslt"] = le.fit_transform(df["rslt"])
    keys = le.classes_
    values = le.transform(le.classes_)
    np.save('labelencoder/rslt.npy', le.classes_)
    dictionary3 = dict(zip(keys, values))
    return df,dictionary1,dictionary2,dictionary3


In [ ]:
# Same for test data.Loading and Encoding.
def label_test_data(df):
    encoder = LabelEncoder()
    encoder.classes_ = np.load('labelencoder/team_abbr.npy',allow_pickle=True)
    df.teamAbbr = encoder.transform(df.teamAbbr)
    encoder = LabelEncoder()
    encoder.classes_ = np.load('labelencoder/oppt_abbr.npy',allow_pickle=True)
    df.opptAbbr = encoder.transform(df.opptAbbr)

    return df


In [ ]:
# For cleaning Nan Values use this function.
def clean_nan_values(df):
    df.dropna(inplace=True)
    return df


In [ ]:
# Scaling All Data and saving standart scaler.

def standart_scaler_all_data(df):
    scaler_filename = "standartScaler/scaler.bin"
    ss = StandardScaler()
    df.iloc[:, 3:] = ss.fit_transform(df.iloc[:, 3:])
    joblib.dump(ss, scaler_filename,compress=True)
    print("Saved Standart Scaler File to ",scaler_filename)
    return df,ss


In [ ]:
# Same for test data load and scale.
def standart_scaler_test_data(df):
    scaler_filename = "standartScaler/scaler.bin"
    scaler = joblib.load(scaler_filename)
    df.iloc[:, 3:] = scaler.transform(df.iloc[:, 3:])
    return df


In [ ]:
# One hot encoding required columns and save it.
def onehotencoder_all_data(df):
    ohe = OneHotEncoder(categorical_features=[0, 1], n_values='auto',
                        handle_unknown='ignore')

    df = ohe.fit_transform(df).toarray()
    df = pd.DataFrame(df)
    onehotencoder_name = "onehotencoder/onehotencoder.bin"
    joblib.dump(ohe, onehotencoder_name, compress=True)
    print("Saved ohe to ",onehotencoder_name)
    return df


In [ ]:
#Same for test data load and encode.
def onehotencoder_test_data(df):
    onehotencoder_name = "onehotencoder/onehotencoder.bin"
    ohe = joblib.load(onehotencoder_name)
    df = ohe.transform(df).toarray()
    return pd.DataFrame(df)


In [ ]:
#You must use this method because some teams is closed in NBA therefore we don't have to use them for training.
def old_to_new_team_abbrs(df):
    currently_available_teams = {}
    currently_available_teams["NJN"] = "BKN" # old to new
    currently_available_teams["NOH"] = "NOP" # old to new

    non_teams = ['EST', 'FLA', 'GNS', 'GUA', 'MAC']
    for key, value in currently_available_teams.items():
        df['teamAbbr'] = df['teamAbbr'].str.replace(key, value)
        df["opptAbbr"] = df["opptAbbr"].str.replace(key, value)
    for each in non_teams:
        drop_index = df[(df.teamAbbr == each) | (df.opptAbbr == each)].index
        df.drop(drop_index, inplace=True)
    print(get_unique_team_abbr(df))
    return df


In [ ]:
#This method is doing same job with re-creating all player stats
#df is main dataframe
#players is player list 
def get_one_shot_player_stats(players,df,last_x_match):
    empty_stats_df = create_empty_player_stats(players)  # create empty stats of teams table
    all_players_stats_df = get_all_players_stats_df(df)  # assign all team stats to empty table
    all_players_stats_last_x_match = get_all_player_stats_last_x_match(all_player_stats_df=all_players_stats_df, all_player_stats_empty_df=empty_stats_df,player_list= players,last_x_match=last_x_match)


In [ ]:
#This one is not include parameter.
#You can use by giving csv urls.
def get_available_player_stats():
    csv_url="../input/matches_w_player_stats.csv"
    df = pd.read_csv(csv_url,header=None)
    df.columns = get_column_names()
    play_by_play = get_play_by_play_stats(df,last_x_match=1000)
    players = pd.read_csv("../input/player_list.csv",header=None)
    players = players.values
    players = players[:,0]
    get_one_shot_player_stats(players,df)


# 2-Building Model

In [ ]:
pip install preprocessing

In [ ]:
#Importing LIBS

from preprocessing import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from xgboost import XGBClassifier,XGBRegressor,Booster
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score,regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import  GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor,RadiusNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor,ExtraTreesRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR,LinearSVR,NuSVR

# 3-Calling Preprocess Methods for our data

In [ ]:
#Preprocessing for predicting match result

def preprocess(csv_url = "/content/inputs/matches_w_player_stats.csv"):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.rslt)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y


# 4-Preprocessing for predicting home and away point


In [ ]:

def preprocess_home_team_point(csv_url ="/content/inputs/matches_w_player_stats.csv"):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.teamPTS)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_away_team_point(csv_url ="/content/inputs/matches_w_player_stats.csv"):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.opptPTS)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y

# 5-Preprocessing for predicting home quarter's point


In [ ]:

def preprocess_home_q1_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.teamPTS1)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_home_q2_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.teamPTS2)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_home_q3_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.teamPTS3)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_home_q4_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.teamPTS4)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y

# 6-Preprocessing for predicting away quarter's point


In [ ]:

def preprocess_away_q1_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.opptPTS1)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_away_q2_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.opptPTS2)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_away_q3_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.opptPTS3)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y
def preprocess_away_q4_point(csv_url):
    df = pd.read_csv(csv_url)  # read data
    df.columns = remove_whitespaces_in_df_columns(df)  # clean column names
    df.columns = get_column_names()  # get column names and assign
    df = old_to_new_team_abbrs(df)
    # players = get_unique_player_list(df)  # get all player list
    # team_abbrs = get_unique_team_abbr(df)  # get all team names
    df = drop_some_columns(df)
    df = clean_nan_values(df)
    Y = np.array(df.opptPTS4)
    encoded,dict1,dict2,dict3 = label_train_data(df)
    scalered ,ss= standart_scaler_all_data(encoded)
    df = onehotencoder_all_data(scalered)
    return df,Y

# 7-Splitting data to Data and Label

In [ ]:
# For this,We use seperate methods to split data because we have 11 model.
def split_data_by_result(df,result_index=60):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    print(Y)
    return X,Y
def split_data_home_point(df, result_index=62):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_away_point(df, result_index=67):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_home_q1_point(df, result_index=63):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_home_q2_point(df, result_index=64):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_home_q3_point(df, result_index=65):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_home_q4_point(df, result_index=66):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_away_q1_point(df, result_index=68):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_away_q2_point(df, result_index=69):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_away_q3_point(df, result_index=70):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y
def split_data_away_q4_point(df, result_index=71):
    X = df.drop(columns=result_index)
    Y = df[result_index]
    return X, Y


## 8-Building Models based XGBOOST

In [ ]:
#Build classification model and save
def build_classifier_model(X_train,Y_train,X_test,Y_test):
    # model = XGBClassifier(n_estimators=5000,nthread=4,seed=42,reg_lambda=0.95,reg_alpha=0.45,tree_method="gpu_hist",max_depth=3,objective="binary:logistic")
    model = XGBClassifier(tree_method="gpu_hist",nthread=4,n_estimators=1000)

    model.fit(X_train, Y_train)
    pickle.dump(model,open("model/xgb_classifier.pkl","wb"))
    print(model.score(X_test,Y_test))
    print(model.classes_)

    return model , model.score(X_test,Y_test)


In [ ]:
def build_class_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess(csv_url)

    X,_ = split_data_by_result(df=df)
    X_train,X_test,Y_train,Y_test = data_split(X,Y)
    model_ , score__ = build_classifier_model(X_train,Y_train,X_test,Y_test)
    return model_,X_train,X_test,Y_train,Y_test


In [ ]:
def build_regressor_model(X_train, Y_train, X_test, Y_test, name):
    # model = XGBRegressor(learning_rate=0.15,gamma=0,reg_lambda=0.01,max_delta_step=0, max_depth=3,n_estimators=10000,
    #                      min_child_weight=1,nthread=4,tree_method="gpu_hist")
    model = XGBRegressor(tree_method="gpu_hist",n_estimators=1000,nthread=4)
    model.fit(X_train, Y_train)
    pickle.dump(model, open(name, "wb"))
    print(model.score(X_test,Y_test))
    return model, model.score(X_test, Y_test)


In [ ]:
def build_home_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_home_team_point(csv_url)
    X,_ = split_data_home_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,
                                             name="model/teams/home_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_away_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_away_team_point(csv_url)
    X,_ = split_data_away_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,name="model/teams/away_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test


In [ ]:
def build_home_q1_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_home_q1_point(csv_url)
    X,_ = split_data_home_q1_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/home_q1_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_home_q2_point_predict_model(csv_url="../input/fulldata/full_data.csv"):
    df,Y=preprocess_home_q2_point(csv_url)
    X,_ = split_data_home_q2_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/home_q2_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_home_q3_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_home_q3_point(csv_url)
    X,_ = split_data_home_q3_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/home_q3_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_home_q4_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_home_q4_point(csv_url)
    X,_ = split_data_home_q4_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/home_q4_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test

In [ ]:
def build_away_q1_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_away_q1_point(csv_url)
    X,_ = split_data_away_q1_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/away_q1_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_away_q2_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_away_q2_point(csv_url)
    X,_ = split_data_away_q2_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/away_q2_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_away_q3_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_away_q3_point(csv_url)
    X,_ = split_data_away_q3_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/away_q3_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test
def build_away_q4_point_predict_model(csv_url="../input/matches_w_player_stats.csv"):
    df,Y=preprocess_away_q4_point(csv_url)
    X,_ = split_data_away_q4_point(df=df)
    X_train,X_test,Y_train,Y_test = data_split_regresyon(X,Y)
    model_ , score__ = build_regressor_model(X_train,Y_train,X_test,Y_test,name="model/teams/away_q4_point_model.pkl")
    return model_,X_train,X_test,Y_train,Y_test

# 9-Getting Prediction with easier way

In [ ]:
def get_result(X, model):
    return model.predict_proba(X)
def get_point_result(X, model):
    return model.predict(X)
def get_points(X,model):
    return model.predict(X) ,model.predict_proba(X)

# 10-Train_Test_Split

In [ ]:
def data_split(X,Y,test_size=0.1):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    Y_train = Y_train.reshape(-1, 1)
    Y_test = Y_test.reshape(-1, 1)
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    return X_train,X_test,Y_train,Y_test
def data_split_regresyon(X,Y,test_size=0.1):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    Y_train = Y_train.reshape(-1, 1)
    Y_test = Y_test.reshape(-1, 1)
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    return X_train,X_test,Y_train,Y_test

# 11-Easier Prediction Functions

In [ ]:
def predict_match_result(data):
    model = pickle.load(open("model/xgb_classifier.pkl","rb"))
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    print(df.head())
    X, Y = split_data_by_result(df=df,result_index=60)
    return get_result(X, model),int(Y[0])


In [ ]:
def predict_home_point(data):
    model = pickle.load(open("model/teams/home_point_model.pkl","rb"))
    Y = np.array(data.teamPTS)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_home_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_away_point(data):
    model = pickle.load(open("model/teams/away_point_model.pkl","rb"))
    Y = np.array(data.opptPTS)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_away_point(df=df)
    return get_point_result(X, model),int(Y[0])

In [ ]:
def predict_home_q1_point(data):
    model = pickle.load(open("model/teams/home_q1_point_model.pkl","rb"))
    Y = np.array(data.teamPTS1)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_home_q1_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_home_q2_point(data):
    model = pickle.load(open("model/teams/home_q2_point_model.pkl","rb"))
    Y = np.array(data.teamPTS2)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_home_q2_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_home_q3_point(data):
    model = pickle.load(open("model/teams/home_q3_point_model.pkl","rb"))
    Y = np.array(data.teamPTS3)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_home_q3_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_home_q4_point(data):
    model = pickle.load(open("model/teams/home_q4_point_model.pkl","rb"))
    Y = np.array(data.teamPTS4)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_home_q4_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_away_q1_point(data):
    model = pickle.load(open("model/teams/away_q1_point_model.pkl","rb"))
    Y = np.array(data.opptPTS1)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_away_q1_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_away_q2_point(data):
    model = pickle.load(open("model/teams/away_q2_point_model.pkl","rb"))
    Y = np.array(data.opptPTS2)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_away_q2_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_away_q3_point(data):
    model = pickle.load(open("model/teams/away_q3_point_model.pkl","rb"))
    Y = np.array(data.opptPTS3)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_away_q3_point(df=df)
    return get_point_result(X, model),int(Y[0])
def predict_away_q4_point(data):
    model = pickle.load(open("model/teams/away_q4_point_model.pkl","rb"))
    Y = np.array(data.opptPTS4)
    encoded= label_test_data(data)
    scalered = standart_scaler_test_data(encoded)
    df = onehotencoder_test_data(scalered)
    X, _ = split_data_away_q4_point(df=df)
    return get_point_result(X, model),int(Y[0])

# 12-You can use build_all_in_one to build all models one line code.

In [ ]:
def build_all_in_one(csv_url="/content/inputs/matches_w_player_stats.csv",test_csv="test_data/GSWvsHOU_test_data.csv"):


    model_, X_train, X_test, Y_train, Y_test = build_class_model(csv_url=csv_url)

    model_, X_train, X_test, Y_train, Y_test = build_home_q1_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_home_q2_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_home_q3_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_home_q4_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_away_q1_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_away_q2_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_away_q3_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_away_q4_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_home_point_predict_model(csv_url=csv_url)
    model_, X_train, X_test, Y_train, Y_test = build_away_point_predict_model(csv_url=csv_url)



In [ ]:
 list_ = get_test_data(teamAbbr="LAC",opptAbbr= "POR", homeplayers=["Caron Butler", "Blake Griffin", "DeAndre Jordan", "Randy Foye",
                         "Chris Paul", "Mo Williams", "Brian Cook", "Ryan Gomes", "Trey Thompkins", "Chauncey Billups",
                         "Eric Bledsoe"],awayplayers=["Caron Butler", "Blake Griffin", "DeAndre Jordan", "Randy Foye", "Chris Paul", "Mo Williams",
                         "Brian Cook", "Ryan Gomes", "Trey Thompkins", "Chauncey Billups", "Eric Bledsoe"],
                          last_x_match=10)

FileNotFoundError: ignored

In [ ]:
os.listdir("test_data")

In [ ]:
build_all_in_one(test_csv="test_data/LACvsPOR_test_data.csv")

# 13-Test our models

In [ ]:
df = pd.read_csv("test_data/LACvsPOR_test_data.csv", index_col=0)
df.drop("index", axis=1, inplace=True)
df.fillna(0, inplace=True)
df_2 = df.copy()
df_3 = df.copy()
df_4 = df.copy()
df_5 = df.copy()
df_6 = df.copy()
df_7 = df.copy()
df_8 = df.copy()
df_9 = df.copy()
df_10 = df.copy()
df_11 = df.copy()
print(predict_match_result(df_11))
print(predict_home_q1_point(df_5))
print(predict_home_q2_point(df_6))
print(predict_home_q3_point(df_7))
print(predict_home_q4_point(df_8))
print(predict_away_q1_point(df))
print(predict_away_q2_point(df_2))
print(predict_away_q3_point(df_3))
print(predict_away_q4_point(df_4))
print(predict_home_point(df_9)) 
print(predict_away_point(df_10))